### 5.1 Dictionary Basics

A **dictionary** is an association between a list of keys and a list of values. Logically it can also be considered as key-value pairs but it is stored physically as a pair of lists.
**Not necessary unique keys!**

Dictionary creation uses the ! operator – read "bang" – in contrast with the syntactic form for lists.

**keys!values**

All dictionaries have type 99h.

In [1]:
10 20 30!1.1 2.2 3.3


10| 1.1
20| 2.2
30| 3.3


A dictionary can be decomposed into its key and value lists using the primitives **key** and **value**. The common number of keys or values is returned by **count**.

In [2]:
d:`a`b`c!100 200 300
key d
value d
count d

`a`b`c


100 200 300


3


When you know that the keys are unique you can apply the `u# attribute to the keys. This will effectively cause the dictionary to be a hash table with the attendant improvement in lookup speed over the default linear lookup.

In [3]:
s:(`u#10 20 30)!1.1 2.2 3.3
s

10| 1.1
20| 2.2
30| 3.3


The order of the items in the key and value lists is significant, just as positional order is significant for lists. Although the I/O assignments and the associated mappings are equivalent regardless of order, differently ordered dictionaries are not identical.

In [6]:
(`a`b`c!10 20 30)~`a`c`b!10 30 20
(`a`b`c!10 20 30)=`a`c`b!10 30 20 / element-wise comparison

0b


a| 1
b| 0
c| 1


#### 5.1.2 Empty and Singleton Dictionaries

##### empty dictionary

In [7]:
()!()  / general empty dictionary

In [9]:
(`symbol$())!`float$() / typed empty disctionary

##### singleton dictionary

In [ ]:
Because both the keys and values are required to be lists, you must enlist atoms for a singleton dictionary.

In [12]:
(enlist `x)!enlist 42  / singleton dictionary
`x!42                  / not a dictionary

x| 42


`x!42


#### 5.1.3 Lookup

In [15]:
d:`a`b`c!10 20 30
d[`b]  / brackets indexing
d `b   / juxtaposition indexing
d[`x]  / lookup of a value that doesn't exist produces null

20


20


0N


In [17]:
d[`a`c]    / indexing is extended item-wise to a simple list of keys.
d (`a; `c)

10 30


10 30


#### 5.1.4 Reverse Lookup with Find ?

? on a dictionary performs reverse lookup by mapping a value item to the first key associated to it.

In [27]:
d:`a`b`c`a!10 20 30 20
d
d?20

a| 10
b| 20
c| 30
a| 20


`b


#### 5.1.5 Dictionary vs. List

In [23]:
L:10 20 30
d:0 1 2!10 20 30
L~d

0b


The two are not the same. A major difference is that dictionaries can be extended via assignment and a list cannot.

#### 5.1.6 Non-unique Keys and Values

In [29]:
ddup:`a`b`a`c!10 20 30 20   / non-unique keys are tolerated
ddup[`a]                    / lookup returns only first value
ddup?20                     / reverse lookup returns only first value
where 20=d                  / list of all keys for given value

10


`b


`b`a


#### 5.1.7 Non-simple Keys and Values

Neither the key nor value lists of a dictionary are required to be atoms or uniform. Either can be nested lists.
Be forewarned that an irregular key or value list – i.e., items do not conform – will confound lookup or reverse lookup, respectively.

In [31]:
dwhackey:(1 2; 3 4 5; 6; 7 8)!10 20 30 40 / atom 6 is whack
dwhackey 1 2

10


In [32]:
dwhackey 6 / returns null as atom 6 doesnt conform

0N


### 5.2 Operations on Dictionaries

#### 5.2.1 Amend and Upsert

As with lists, the items of a dictionary can be modified via assignment to a key.

In [33]:
d:`a`b`c!10 20 30
d[`b]:42            / amend

**In contrast to lists, dictionaries can be extended via assignment.**

In [35]:
d[`x]:66
d

a| 10
b| 42
c| 30
x| 66


**d[k]:v** <br>
It updates the existing key-value association when k is in key d, or inserts (appends) a new key-value association when k is not in key d.
This update/insert behavior is called **upsert** semantics. Because tables and keyed tables are dictionaries, upsert semantics pervade kdb+.

#### 5.2.2 Extracting a Sub-Dictionary

Dictionary lookup on a key or a list of keys returns the associated values. It is possible to extract both keys and values using an overload of the **take operator #**. The left operand is a list of keys, the right operand is the source dictionary and the result is a sub-dictionary for the specified keys. 

In [37]:
d:`a`b`c!10 20 30
vlist:d `a`c   / list of values corresponding to keys
vlist

subd:`a`c#d    / subdictionary
subd

10 30


a| 10
c| 30


In the event of duplicate keys, only the first is extracted. <br>
This operation works with non-simple keys.

#### 5.2.3 Removing Entries

The adjoint operation to # is **_**, which removes key-value pairs. Its syntax is the same – i.e., a list of keys as the left operand and a dictionary on the right – and it returns the dictionary obtained by removing the key-value pairs for the specified keys.

In [41]:
d:`a`b`c!10 20 30
`a`c _ d
`b _ d
`a`c cut d    / cut is same as _
-3!`a`b`c _ d / empty typed dictionary


b| 20


a| 10
c| 30


b| 20


"(`symbol$())!`long$()"


#### 5.2.4 Basic Operations on Dictionaries

##### Monadic

 applying a function to a dictionary effectively applies it to the value list.

In [44]:
d:`a`b`c!10 20 30
neg d
2*d
d=20

a| -10
b| -20
c| -30


a| 20
b| 40
c| 60


a| 0
b| 1
c| 0


In [45]:
f:{x*x}
f d

a| 100
b| 400
c| 900


##### Dyadic

When the domains ( lists of keys ) of two dictionary maps are identical, performing a dyadic operation is also straightforward: do it value-with-value along the keys. For example, to add two dictionaries with a common list of keys, add their corresponding values.

In [47]:
d1:`a`b`c!1 2 3
d2:`a`b`c!10 20 30
d1+d2

a| 11
b| 22
c| 33


When the domains ( lists of keys ) of two dictionary maps are different, 
- for common keys dyadic function will be applied to corresponding values
- for missed keys value the only existing value will be used

In [48]:
d1:`a`b`d!1 2 3
d2:`a`b`c!10 20 30
d1+d2

a| 11
b| 22
d| 3
c| 30


#### 5.2.5 Join ,

**The join operator ,** merges two dictionaries. 

When no common keys:

In [49]:
d1:`a`b`c!10 20 30
d2:`x`y!40 50
d1,d2

a| 10
b| 20
c| 30
x| 40
y| 50


When there are common keys, , since q is right-to-left, **the value of the right operand prevails over that of the left.** <br>
Entire right operand is upserted into the left operand. <br>
 join is not commutative: order matters.

In [50]:
d1:`a`b`c!10 20 30
d2:`a`b`c!100 200 300
d1,d2

a| 100
b| 200
c| 300


#### 5.2.6 Coalesce ^

The coalesce operator is related to , in that it employs upsert semantics to merge two dictionaries. **The difference with , is that right values prevail over left except for nulls in the right.**

In [51]:
d1:`a`b`c!10 0N 30
d2:`b`c`d!200 0N 400
d1^d2

a| 10
b| 200
c| 30
d| 400


#### 5.2.7 Arithmetic And Relational Operations

Following the general pattern, when an arithmetic operation is performed on dictionaries, the operation is performed on the common domain elements and the identity element implied elsewhere. Of course the value lists must have proper types for the operation.

For equality and comparison operations on dictionaries, the indicated operation is performed over the common keys. On disjoint keys, the appropriate null is effectively substituted for missing values. Due to all nulls being equal, a null value on a disjoint key ( key present only in 1 of 2 dictionaries ) will report 1b under equality test.

In [52]:
(`a`b`c!10 20 30)=`b`c`d!20 300 400

a| 0
b| 1
c| 0
d| 0


In [53]:
(`a`b`c!0N 20 30)=`b`c`d!20 300 0N / key `a is disjoint with null value in left dict

a| 1
b| 1
c| 0
d| 1


### 5.3 Column Dictionaries

**Column dictionary** is dictionary which maps simple list of symbols to a rectangular list of lists.

Interpreting each symbol as a name and the corresponding list as a vector of column values, we call such a list a column dictionary. A general column dictionary has the form,

c1...cn!(v1;...;vn)

where each ci is a symbol and the vi are lists with common length (count). Such a dictionary associates the name ci with the value list vi.


In [6]:
travelers:`name`iq!(`Dent`Beeblebrox`Prefect;42 98 126)
travelers
travelers[`name]
travelers[`name][1]
travelers[`name; 1]    /    indexing-at-depth
travelers[; 2]         /    slicing, returns dictionary 

name| Dent Beeblebrox Prefect
iq  | 42   98         126    


`Dent`Beeblebrox`Prefect


`Beeblebrox


`Beeblebrox


name| `Prefect
iq  | 126


To summarize.

- A column dictionary can be viewed as a two-dimensional entity with retrieval by name in the first dimension and by position within column in the second.
- Specifying only a name retrieves the corresponding column, which can then be indexed by position.
- Specifying only a column index retrieves a slice dictionary that maps the names to the associated values across that column index.

This data structure has potential but there is a hitch: the indexing is backwards. That is, columns are retrieved in the first slot. Normally in two-dimensional data structures, columns are retrieved in the second slot.

In [9]:
`c!1 2 3
(enlist `c)!enlist 1 2 3  / second enlist produces list of lists, each with 1 element

`c!1 2 3


c| 1 2 3


In [25]:
d:enlist 10 20 30   / workd for each atom in list
d[0]
d[0][0]
d[0][1]


10 20 30


10


20


10 20 30


20


### 5.4 Flipping a Column Dictionary

In [26]:
dc:`c1`c2!(`a`b`c; 10 20 30)
dc

c1| a  b  c 
c2| 10 20 30


In [27]:
t:flip dc
t

c1 c2
-----
a  10
b  20
c  30


In [28]:
dc = flip flip dc

c1| 111b
c2| 111b


In [29]:
dc[`c1;1]
t[1;`c1]

`b


`b


In [31]:
t[0]
t[1]
t[2]

c1| `a
c2| 10


c1| `b
c2| 20


c1| `c
c2| 30


We summarize our findings about the transpose of a column dictionary,

- It is a two-dimensional data structure that uses an integer index in the first slot and a column name symbol in the second slot.
- Specifying only an integer in the first slot retrieves a section dictionary across that index
- Specifying only a column name in the second slot retrieves that column.
- Specifying both an integer and a column name retrieves the "field" at that index in that column.
- It can be viewed as a list of section dictionaries.

Unlike the case of transposing rectangular lists, transposing a column dictionary does not physically re-arrange data.
A transposed column dictionary is stored in column order.

Instead, the result is a logical adjustment – meaning that the slots in indexing at depth are reversed.